In [70]:
# Ici on compare la méthode implémentée sur tensorflow à la méthode déja existante de la library kornia

# Kornia : 


import torch
import tensorflow as tf

def fundamental_from_projections(P1: torch.Tensor, P2: torch.Tensor) -> torch.Tensor:
    r"""Get the Fundamental matrix from Projection matrices.

    Args:
        P1: The projection matrix from first camera with shape :math:`(*, 3, 4)`.
        P2: The projection matrix from second camera with shape :math:`(*, 3, 4)`.

    Returns:
         The fundamental matrix with shape :math:`(*, 3, 3)`.

    """
    def vstack(x, y):
        return torch.cat([x, y], dim=-2)
    X1 = P1[..., 1:, :]
    X2 = vstack(P1[..., 2:3, :], P1[..., 0:1, :])
    X3 = P1[..., :2, :]

    Y1 = P2[..., 1:, :]
    Y2 = vstack(P2[..., 2:3, :], P2[..., 0:1, :])
    Y3 = P2[..., :2, :]

    X1Y1, X2Y1, X3Y1 = vstack(X1, Y1), vstack(X2, Y1), vstack(X3, Y1)
    X1Y2, X2Y2, X3Y2 = vstack(X1, Y2), vstack(X2, Y2), vstack(X3, Y2)
    X1Y3, X2Y3, X3Y3 = vstack(X1, Y3), vstack(X2, Y3), vstack(X3, Y3)
    F_vec = torch.cat(
        [
            X1Y1.det().reshape(-1, 1),
            X2Y1.det().reshape(-1, 1),
            X3Y1.det().reshape(-1, 1),
            X1Y2.det().reshape(-1, 1),
            X2Y2.det().reshape(-1, 1),
            X3Y2.det().reshape(-1, 1),
            X1Y3.det().reshape(-1, 1),
            X2Y3.det().reshape(-1, 1),
            X3Y3.det().reshape(-1, 1),
        ],
        dim=1,
    )
    print(*P1.shape[:-2])
    print( F_vec.view(*P1.shape[:-2], 3, 3))
    return F_vec.view(*P1.shape[:-2], 3, 3)


In [90]:
# Test
P2t=torch.tensor([[162.36,-438.34,-17.508,3347.4],[73.3,-10.043,-443.34,1373.5],[0.99035,-0.047887,-0.13009,6.6849]])
P1t=torch.tensor([[439.0,180.81,-26.946,185.95],[-5.3416,88.523,-450.95,1324],[0.0060594,0.99348,-0.11385,5.227]])
ft=fundamental_from_projections(P1t,P2t)


tensor([[-1.2074e+04,  1.5901e+06, -1.4935e+08],
        [ 1.9150e+06,  6.7462e+04,  3.0550e+08],
        [-1.5098e+08, -1.1021e+09,  7.7577e+10]])


In [86]:
# Notre méthode
def fundamental_matrix_from_projections(P1, P2):
    """
    Get the Fundamental matrix from Projection matrices.
    Adapted from 
    [
    https://kornia.readthedocs.io/en/latest/_modules/kornia/geometry/epipolar
    /_metrics.html#sampson_epipolar_distance
    ]
    """
    def vstack(x, y):
            return tf.concat([x,y], axis=0, name='concat')
    X1 = P1[..., 1:, :]
    X2 = vstack(P1[..., 2:3, :], P1[..., 0:1, :])
    X3 = P1[..., :2, :]

    Y1 = P2[..., 1:, :]
    Y2 = vstack(P2[..., 2:3, :], P2[..., 0:1, :])
    Y3 = P2[..., :2, :]

    X1Y1, X2Y1, X3Y1 = vstack(X1, Y1), vstack(X2, Y1), vstack(X3, Y1)
    X1Y2, X2Y2, X3Y2 = vstack(X1, Y2), vstack(X2, Y2), vstack(X3, Y2)
    X1Y3, X2Y3, X3Y3 = vstack(X1, Y3), vstack(X2, Y3), vstack(X3, Y3)
    F_vec = tf.concat(
        [
            tf.reshape(tf.linalg.det(X1Y1),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X2Y1),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X3Y1),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X1Y2),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X2Y2),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X3Y2),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X1Y3),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X2Y3),shape=(-1,1)),
            tf.reshape(tf.linalg.det(X3Y3),shape=(-1,1)),
        ],
        axis=-1
    )

    return tf.reshape(F_vec,shape=(*P1.shape[:-2],3,3))


In [87]:

P2=tf.constant([[162.36,-438.34,-17.508,3347.4],[73.3,-10.043,-443.34,1373.5],[0.99035,-0.047887,-0.13009,6.6849]])
P1=tf.constant([[439.0,180.81,-26.946,185.95],[-5.3416,88.523,-450.95,1324],[0.0060594,0.99348,-0.11385,5.227]])
fundamental_matrix_from_projections(P1,P2)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-1.20740205e+04,  1.59010288e+06, -1.49354128e+08],
       [ 1.91498150e+06,  6.74619453e+04,  3.05496512e+08],
       [-1.50978688e+08, -1.10211149e+09,  7.75774781e+10]], dtype=float32)>

In [98]:
# Version kornia

def symmetrical_epipolar_distance(
    pts1: torch.Tensor, pts2: torch.Tensor, Fm: torch.Tensor, squared: bool = True, eps: float = 1e-8
) -> torch.Tensor:
    r"""Return symmetrical epipolar distance for correspondences given the fundamental matrix.

    Args:
       pts1: correspondences from the left images with shape
         (B, N, 2 or 3). If they are not homogeneous, converted automatically.
       pts2: correspondences from the right images with shape
         (B, N, 2 or 3). If they are not homogeneous, converted automatically.
       Fm: Fundamental matrices with shape :math:`(B, 3, 3)`. Called Fm to
         avoid ambiguity with torch.nn.functional.
       squared: if True (default), the squared distance is returned.
       eps: Small constant for safe sqrt.

    Returns:
        the computed Symmetrical distance with shape :math:`(B, N)`.

    """
    if not isinstance(Fm, torch.Tensor):
        raise TypeError(f"Fm type is not a torch.Tensor. Got {type(Fm)}")

    if (len(Fm.shape) != 3) or not Fm.shape[-2:] == (3, 3):
        raise ValueError(f"Fm must be a (*, 3, 3) tensor. Got {Fm.shape}")

    if pts1.size(-1) == 2:
        pts1 = convert_points_to_homogeneous(pts1)

    if pts2.size(-1) == 2:
        pts2 = convert_points_to_homogeneous(pts2)

    # From Hartley and Zisserman, symmetric epipolar distance (11.10)
    # sed = (x'^T F x) ** 2 /  (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))

    # line1_in_2: torch.Tensor = (F @ pts1.permute(0,2,1)).permute(0,2,1)
    # line2_in_1: torch.Tensor = (F.permute(0,2,1) @ pts2.permute(0,2,1)).permute(0,2,1)

    # Instead we can just transpose F once and switch the order of multiplication
    F_t: torch.Tensor = Fm.permute(0, 2, 1)
    line1_in_2: torch.Tensor = pts1 @ F_t
    line2_in_1: torch.Tensor = pts2 @ Fm

    # numerator = (x'^T F x) ** 2
    numerator: torch.Tensor = (pts2 * line1_in_2).sum(2).pow(2)

    # denominator_inv =  1/ (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))
    denominator_inv: torch.Tensor = 1.0 / (line1_in_2[..., :2].norm(2, dim=2).pow(2)) + 1.0 / (
        line2_in_1[..., :2].norm(2, dim=2).pow(2)
    )
    out: torch.Tensor = numerator * denominator_inv
    if squared:
        return out
    return (out + eps).sqrt()


In [120]:
Point_2t=torch.tensor([1.,2.,3.])
Point_1t=torch.tensor([4.,5.,5.])
fmt=torch.tensor([[[1,2,3],[2,3,4],[2,3,4]],[[15,6,6],[2,3,4],[5,6,2]]])
symmetrical_epipolar_distance(Point_1t,Point_2t, fmt, squared=True, eps= 1e-8)

RuntimeError: expected scalar type Long but found Float

In [ ]:
# Version tensoflow

def tf_symmetrical_epipolar_distance(pts1,pts2,Fm, squared, eps) : 
    """
    Return symmetrical epipolar distance for correspondences given the fundamental matrix.

    """
    
    if not isinstance(Fm, tf.Tensor):
        raise TypeError(f"Fm type is not a torch.Tensor. Got {type(Fm)}")

    if (len(Fm.shape) != 3) or not Fm.shape[-2:] == (3, 3):
        raise ValueError(f"Fm must be a (*, 3, 3) tensor. Got {Fm.shape}")

        ###########Acompleter
 #   if pts1.size(-1) == 2:
 #       pts1 = convert_points_to_homogeneous(pts1)

 #   if pts2.size(-1) == 2:
  #      pts2 = convert_points_to_homogeneous(pts2)

    # From Hartley and Zisserman, symmetric epipolar distance (11.10)
    # sed = (x'^T F x) ** 2 /  (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))
    # Instead we can just transpose F once and switch the order of multiplication
    
    F_t = tf.transpose(Fm, perm=(0,2,1), conjugate=False, name='permute')
    line1_in_2 = pts1 @ F_t
    line2_in_1 = pts2 @ Fm

    # numerator = (x'^T F x) ** 2
    #numerator  = (pts2 * line1_in_2).sum(2).pow(2)
    numerator  = tf.pow(tf.math.reduce_sum((pts2 * line1_in_2),2),2)


    # denominator_inv =  1/ (((Fx)_1**2) + (Fx)_2**2)) +  1/ (((F^Tx')_1**2) + (F^Tx')_2**2))
    denominator_inv = 1.0 / (line1_in_2[..., :2].norm(2, dim=2).pow(2)) + 1.0 / (
        line2_in_1[..., :2].norm(2, dim=2).pow(2)
    )
    
    denominator_inv = 1.0 / (tf.pow(tf.norm(line1_in_2[..., :2],axis=2),2)) + 1.0 / (
        tf.pow(tf.norm(line2_in_1[..., :2],axis=2),2)
    )
    
    out = numerator * denominator_inv
    if squared:
        return out
    return tf.math.sqrt(out + eps)


In [ ]:
# version kornia
def convert_points_to_homogeneous(points):
    r"""Function that converts points from Euclidean to homogeneous space.

    See :class:`~torchgeometry.ConvertPointsToHomogeneous` for details.

    Examples::

        >>> input = torch.rand(2, 4, 3)  # BxNx3
        >>> output = tgm.convert_points_to_homogeneous(input)  # BxNx4
    """
    if not torch.is_tensor(points):
        raise TypeError("Input type is not a torch.Tensor. Got {}".format(
            type(points)))
    if len(points.shape) < 2:
        raise ValueError("Input must be at least a 2D tensor. Got {}".format(
            points.shape))

    return nn.functional.pad(points, (0, 1), "constant", 1.0)